In [3]:


import numpy as np
import pandas as pd

from IPython.display import display

from ipywidgets import interact
import ipywidgets as wid

import itertools



tree = [[],
        ["Китай", "Южная Корея"],
        ["Сериал", "Полнометражный", "Мини-сериал"],
        ["JSTV", "Tencent Video", "Mango TV", "Hunan TV", "Dragon TV", "SBS", "KBS2", "OCN", "tvN", "MBC",
         "Naver TV Cast", "-"]]

weights = [.5, .7, .1]


categories = ['Сериал', 'Полнометражный', 'Мини-сериал']

categoriesMatrix = [
    [1, 0, 0.5],
    [0, 1, 0],
    [0.5, 0, 1],
]



def diff_tree(t1, t2):
    subtree1 = t1
    subtree2 = t2
    difftree = []
    for i in range(1, 4):
        difftree.append(abs(tree[i].index(subtree1[i]) - tree[i].index(subtree2[i])))

    similarity = 0
    for i in range(len(difftree)):
        similarity += difftree[i] * weights[i]

    return similarity

def manhattanDistance(a, b):
    D = 0
    for i in range(len(a)):
        D += abs(a[i] - b[i])
    return D

def create_matrix(df, metric):
    matr = []
    for i in range(len(df.values.tolist())):
        r = []
        for k in range(len(df.values.tolist())):
            r.append(metric(df.values.tolist()[i], df.values.tolist()[k]))
        matr.append(np.array(r))

    return np.array(matr)

def tree_and_menhetten(df, dfFull):
    # tree = create_matrix(dfFull, diff_tree)
    m = create_matrix(df, manhattanDistance)
    # result = 1.5*tree + m
    return m

def show_matrix(matr):
    plt.imshow(matr)
    plt.colorbar()
    
    plt.xticks(np.arange(0, len(df.values.tolist())), rotation=-45)
    plt.yticks(np.arange(0, len(df.values.tolist())))
    figure(figsize=(20, 20), dpi=160)
    plt.show()

def get_similars_from_matrix(dfFull, matr, id):
     return pd.DataFrame(list(zip(matr[id], map(lambda e: str(" ".join(e[:4])), dfFull.values.tolist()))),
                        index=np.arange(len(matr[id])), columns=['Степень различия', 'тип'])
     

def getSimilarsByGroupID(dfFull, similarsMatrix, likeIDs, disIDs):
    if (len(likeIDs) == 0 and len(disIDs) == 0):
      return dfFull
    
    disliked_result = []
    for id in disIDs:
        res = get_similars_from_matrix(dfFull, similarsMatrix, id)
#         print(res.sort_values('Степень различия')[: 5])
        if len(disliked_result) == 0: 
            disliked_result = res['Степень различия']
        else:
            for i in range(len(disliked_result)):
                disliked_result[i] = disliked_result[i] if disliked_result[i] < res['Степень различия'][i] else res['Степень различия'][i]
    # print('dislikes')
    # print(disliked_result)

    liked_result = []
    for id in likeIDs:
        res = get_similars_from_matrix(dfFull, similarsMatrix, id)
        
        if len(liked_result) == 0: 
            liked_result = res['Степень различия']
        else:
            for i in range(len(liked_result)):
                liked_result[i] = liked_result[i] if liked_result[i] < res['Степень различия'][i] else res['Степень различия'][i]
    # print('likes')
    # print(liked_result)

    result = liked_result
    if len(disliked_result) > 0:
        disliked_max = max(disliked_result.values)
        disliked_result = disliked_result * (-1) + disliked_max
        # print('dislikes')
        # print(disliked_result)

        result = liked_result + disliked_result
    # print('result')
    # print(result)

    return pd.DataFrame(list(map(lambda x: [x[0]] + x[1], list(zip(result, dfFull.values.tolist())))),
                        index=np.arange(len(result)), columns=['Степень различия'] + list(dfFull.columns.values))



dfFull = pd.read_csv('./dataset.txt', delimiter=',', encoding="utf8")


df = dfFull.copy(deep=True)

df["Жанр"] = df["Жанр"].map(lambda e: str(e).split(','))
df["Теги"] = df["Теги"].map(lambda e: str(e).split(','))
df["Актеры"] = df["Актеры"].map(lambda e: str(e).split(','))

genres = list(set(itertools.chain.from_iterable(df["Жанр"].values)))

tags = list(set(itertools.chain.from_iterable(df["Теги"].values)))

actors = list(set(itertools.chain.from_iterable(df["Актеры"].values)))

chanels = list(set(df["Канал"].values))

titles = list(df["Название"].values)

produsers = list(set(df['Режиссер'].values))

print(produsers)


ndf = df.copy() 

del ndf["Название"]

ndf["Страна"], countries_unics = pd.factorize(ndf["Страна"])
ndf["Канал"], chanels_unics = pd.factorize(ndf["Канал"])
max_chaneles = max(ndf["Канал"].values)
ndf["Канал"] = ndf["Канал"].values / max(ndf["Канал"].values)
ndf["Озвучка"], _ = pd.factorize(ndf["Озвучка"])
ndf["Режиссер"], _  = pd.factorize(ndf["Режиссер"])

ndf["Год"] = ndf["Год"].map(lambda e: float(e))
years_diff = (max(ndf["Год"].values) - min(ndf["Год"].values))
years_min = min(ndf["Год"].values)
ndf["Год"] = (ndf["Год"].values - min(ndf["Год"].values)) / (max(ndf["Год"].values) - min(ndf["Год"].values))

ndf["Рейтинг"] = ndf["Рейтинг"].map(lambda e: float(e))
rating_max =  max(ndf["Рейтинг"].values)
ndf["Рейтинг"] = ndf["Рейтинг"].values / max(ndf["Рейтинг"].values)
ndf["Количество серий"] = ndf["Количество серий"].map(lambda e: float(e))
series_count_max = max(ndf["Количество серий"].values)
ndf["Количество серий"] = ndf["Количество серий"].values / max(ndf["Количество серий"].values)

for genre in genres:
  ndf[genre] = ndf["Жанр"].map(lambda e: 1 if (genre in e) else 0)

for tag in tags:
  ndf[tag] = ndf["Теги"].map(lambda e: 1 if (tag in e) else 0)

for actor in actors:
  ndf[actor] = ndf["Актеры"].map(lambda e: 1 if (actor in e) else 0)

for i in range(len(categories)):
  ndf[categories[i]] = df['Категория'].map(lambda e: categoriesMatrix[i][categories.index(e)])

del ndf["Жанр"]
del ndf['Теги']
del ndf['Категория']
del ndf["Режиссер"]
del ndf["Актеры"]




['Цзинь Ди Жун', 'Син Ён Хви', 'Цю Чжун Вэй', 'Пак Сын У', 'Пак Дан Хи', 'Ким Пён Су', 'У Цзинь Юань', ': Чжан Кай Чжоу', 'Линь Юй Фэнь', 'Гао И Цзюнь', 'Ким Гю Тхэ', 'Ким Джи Хён', 'Чо Су Вон', 'Квон О Сын', 'Ли Сон Джун', 'Квон Джин Мо', 'Мин Янь', 'Ким Чон Хак', 'Ким Ён Хва', 'О Чун Хван', '\r\nВан Чжи', 'Ким Хон Сон', 'Ли Чон Соп']


In [4]:
@interact(
    
    query = wid.Text(
    value='',
    placeholder='Введите что-нибудь...',
    description='Запрос:',
    disabled=False
),
    genre= wid.SelectMultiple(
    options=genres,
    value=[],
    rows=10,
    description='Жанр',
    disabled=False),

    actor= wid.SelectMultiple(
    options=actors,
    value=[],
    rows=5,
    description='Актеры',
    disabled=False),

    title= wid.SelectMultiple(
    options=titles,
    value=[],
    rows=5,
    description='Отсортировать по похожести на: ',
    disabled=False),

    produser= wid.SelectMultiple(
    options=produsers,
    value=[],
    rows=5,
    description='Режиссер',
    disabled=False),
    
    countries= wid.SelectMultiple(
    options=['Китай', 'Южная Корея'],
    value=[],
    rows=2,
    description='Страна',
    disabled=False),
    
    category= wid.SelectMultiple(
    options=['Сериал', 'Мини-сериал','Полнометражный'],
    value=[],
    rows=3,
    description='Категория',
    disabled=False),
    
    chanel= wid.SelectMultiple(
    options=chanels,
    value=[],
    rows=3,
    description='Канал',
    disabled=False),
  
    
    years = wid.IntRangeSlider(
    value=[2006, 2021],
    min=2006,
    max=2021,
    step=1,
    description='Года выпуска:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
),
    
        
    rating = wid.IntSlider(
    value=0,
    min=0,
    max=5,
    step=1,
    description='Рейтинг от:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'),
    
    series_count = wid.IntRangeSlider(
    value=[1, 100],
    min=1,
    max=100,
    step=1,
    description='Количесвто серий:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=wid.Layout(width='300px')
    ),
    onlyWithVoice = wid.Checkbox(
    value=False,
    description='Только с озвучкой',
    disabled=False,
    indent=False
),
tag=wid.SelectMultiple(
    options=tags,
    value=[],
    rows=15,
    description='Теги',
    disabled=False),
)
def search(
    title=[],
        query='',
        countries=[],
        category=[],
        genre=[],
        actor=[],
        produser=[],
    chanel=[],
    years=[2000, 2021],
    onlyWithVoice=False,
    rating=0,
    series_count=[],
    tag=[]
):
    result = df.copy(deep=True)
    
    frame = ndf.copy()
    dfFullCopy = dfFull.copy()
    resultMatrix = tree_and_menhetten(frame, dfFullCopy)
    if (len(title) > 0):
      title2 = [titles.index(t) for t in title ]
      result = getSimilarsByGroupID(result, resultMatrix, title2, [])
      result = result.sort_values('Степень различия')
      result = result[~(result['Название']).isin(title)]
    if (len(countries) > 0):
      result = result[(result['Страна']).isin(countries)]
    
    if (len(category) > 0):
      result = result[(result['Категория']).isin(category)]

    if (len(chanel) > 0):
      result = result[result['Канал'].isin(chanel)]

    if (len(produser) > 0):
      result = result[result['Режиссер'].isin(produser)]

    if (len(genre) > 0):
      result = result[result.apply(lambda x:bool(set(x['Жанр']) & set(genre)) , axis=1)]

    if (len(actor) > 0):
      result = result[result.apply(lambda x:bool(set(x['Актеры']) & set(actor)) , axis=1)]

    if (len(tag) > 0):
      result = result[result.apply(lambda x:bool(set(x['Теги']) & set(tag)) , axis=1)]

  
    result = result.loc[result['Год'] > years[0]]
    result = result.loc[result['Год'] < years[1]]
    result = result[result['Рейтинг'] > rating]
    
    result = result.loc[result['Количество серий'] > series_count[0]]
    result = result.loc[result['Количество серий'] < series_count[1]]
    if onlyWithVoice:
      result = result[result['Озвучка'] == 'Да']

    if len(query) > 0: 
      result = result[result['Название'].str.contains(query)]

    if len(result) > 0: 
      display(result)
    else: 
      
      rowcopy = ndf.loc[0].copy()

      for column in ndf: 
        # print(rowcopy[column])
        rowcopy[column] = 0

      rowcopy["Год"] = (years[1] + years[0])/2 
      rowcopy['Год'] = (rowcopy["Год"] - years_min) / years_diff

      chanels_diff = 0 
      for c in chanel: 
        chanels_diff += (list(chanels_unics)).index(c)
      chanels_diff = chanels_diff/max_chaneles

      rowcopy["Канал"] = chanels_diff
      rowcopy["Озвучка"] = 1 if onlyWithVoice else 0
      rowcopy["Рейтинг"] = rating/5

      rowcopy["Количество серий"] = (series_count[1] + series_count[0]) / 2 / max(dfFull["Количество серий"].values)

        
      for g in genres:
        rowcopy[g] = 1 if g in genre else 0

      for t in tags:
        rowcopy[t] = 1 if t in tag else 0

      for a in actors:
        rowcopy[a] = 1 if a in actor else 0

      for c in categories:
        rowcopy[c] = 1 if c in category else 0

      frame = ndf.copy()
      frame.append(rowcopy)
      frame.loc[len(frame)] = rowcopy 
      # display(frame)

      dfFullCopy = dfFull.copy()
      dfFullCopy.loc[len(dfFullCopy)] =  dfFull.loc[0].copy()
            
      resultMatrix = tree_and_menhetten(frame, dfFullCopy)

      result = get_similars_from_matrix(dfFullCopy, resultMatrix, len(frame) - 1 )
      result = result.sort_values('Степень различия')
       
      # result = result.loc[1:]
      print("К сожалению, по вашему запросу ничего не найдено.") 
      print("Возможно, вам понравится:")
      i = 0
      for r in result['тип']:
        
        if (i != 0 and i < 6):
          print(i, r)

        i += 1



interactive(children=(SelectMultiple(description='Отсортировать по похожести на: ', options=('Лёгкая улыбка по…